In [1]:
import pandas as pd

### Extract data into pandas DataFrame objects

In [2]:
# specify the S3 bucket where we've published the government and API data
s3_bucket = "quest-szlr1n1km8kc"
csv_path = f"s3://{s3_bucket}/pr.data.0.Current"
json_path = f"s3://{s3_bucket}/api_data.json"

In [53]:
pop_df = pd.read_json(path_or_buf=json_path)
pop_df = pop_df.rename(columns=lambda x: x.strip().lower())
pop_df

,id nation,nation,id year,year,population,slug nation
0,01000US,United States,2022,2022,331097593,united-states
1,01000US,United States,2021,2021,329725481,united-states
2,01000US,United States,2020,2020,326569308,united-states
3,01000US,United States,2019,2019,324697795,united-states
4,01000US,United States,2018,2018,322903030,united-states
5,01000US,United States,2017,2017,321004407,united-states
6,01000US,United States,2016,2016,318558162,united-states
7,01000US,United States,2015,2015,316515021,united-states
8,01000US,United States,2014,2014,314107084,united-states
9,01000US,United States,2013,2013,311536594,united-states


In [75]:
df = pd.read_csv(filepath_or_buffer=csv_path, sep='\t')

# clean up
df = df.rename(columns=lambda x: x.strip().lower())
for col in ['series_id', 'period']:
    df[col] = df[col].str.strip()
df

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
35992,PRS88003203,2023,Q02,116.953,NaN
35993,PRS88003203,2023,Q03,116.928,NaN
35994,PRS88003203,2023,Q04,115.824,NaN
35995,PRS88003203,2023,Q05,116.472,NaN


### Generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive

In [54]:
mean = pop_df['population'].where(pop_df['year'].between(2013, 2018, inclusive='both')).mean()
print(f"Mean population: {mean:.2f}")

Mean population: 317437383.00


In [55]:
std_dev = pop_df['population'].where(pop_df['year'].between(2013, 2018, inclusive='both')).std()
print(f"Standard deviation: {std_dev:.2f}")

Standard deviation: 4257089.54


### Find the best year

In [66]:
# group the rows by series_id and year to get a sum for each group
best_year_df = df.groupby(['series_id', 'year']).agg({'value': 'sum'}).reset_index()

# for each series_id we select the maximum sum value
best_year_df = best_year_df.loc[best_year_df.groupby('series_id')['value'].idxmax()].reset_index(drop=True)

best_year_df

,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2023,767.007
279,PRS88003201,2022,36.100
280,PRS88003202,2022,28.900


### Find the value for `series_id = PRS30006032` and `period = Q01` and the population value for that given year

In [85]:
# filter the CSV DataFrame for the specific series_id and period
filtered_df = df[(df['series_id'] == 'PRS30006032') & (df['period'] == 'Q01')]

# merge with the population DataFrame to get the `population` column
report_df = filtered_df.merge(pop_df, on='year', how='left')

# display result in example format
display_cols = ['series_id', 'year', 'period', 'value', 'population']
report_df[display_cols]

,series_id,year,period,value,population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.4,NaN
2,PRS30006032,1997,Q01,2.7,NaN
3,PRS30006032,1998,Q01,1.0,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.5,NaN
7,PRS30006032,2002,Q01,-6.7,NaN
8,PRS30006032,2003,Q01,-5.6,NaN
9,PRS30006032,2004,Q01,2.1,NaN
